In [1]:
import cv2
import pytesseract
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
pd.set_option('display.max_rows', None)

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"
        
def read_image(image):
    ##Preprocessing the image
    #Cropping the image so that it only shows the Student ID number. This may be required to be adjusted for each user.
    cropped = image[120:1032, 60:127] 

    #Gray scaling the image.
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    #Threshholding the image.
    thresh = cv2.threshold(gray, 0, 220, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    #Student ID number detection goes on here. The configuration allows for it to detect only numbers.
    result = pytesseract.image_to_string(thresh ,config='--psm 4 --oem 3 -c tessedit_char_whitelist=0123456789')
    result = result.split()
    return result

def update_attendance(file_name):
    attendanceSheet = pd.read_excel(file_name)
    attendance = attendanceSheet['StudentID'].apply(lambda x: any(item for item in res if item == x ))
    attendanceSheet['Present'] = 'X'
    attendanceSheet.Present[attendance] = "O"
    today = pd.to_datetime('today').strftime("%m/%d/%Y")
    attendanceSheet = attendanceSheet.rename(columns = {'Present':today})
    return attendanceSheet

In [3]:
my_button = SelectFilesButton()
my_button2 = SelectFilesButton()

In [4]:
button_send = widgets.Button(
                description='Take Attendance',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()

def on_button_clicked(event):
    with output:
        results = []
        for x in range(len(my_button.files)):
            img = cv2.imread(my_button.files[x])
            results += read_image(img) 
        res = []
        for i in results:
            if i not in res:
                res.append(i)
                res = list(map(int, res))
        attendanceSheet = pd.read_excel(my_button2.files[0])
        attendance = attendanceSheet['StudentID'].apply(lambda x: any(item for item in res if item == x ))
        attendanceSheet['Present'] = 'X'
        attendanceSheet.Present[attendance] = "O"
        today = pd.to_datetime('today').strftime("%m/%d/%Y")
        attendanceSheet = attendanceSheet.rename(columns = {'Present':today})
        display(attendanceSheet)

button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

In [5]:
text_0 = widgets.HTML(value="<h1>Automated Attendance Serivce</h1>")
text_1 = widgets.HTML(value="<h4>Please select and upload image file.</h4>")
text_2= widgets.HTML(value="<h4>Please select and upload attendance file.</h2>")
#text_3= widgets.HTML(value="<h2>for dinner, if that's okay for you!</h2>")
#text_4= widgets.HTML(value="<h2>Also, if you don't mind, I'll bring along </h2>")
#text_5= widgets.HTML(value="<h2>hungry ghosts for dinner, if that's okay for you!</h2>")

In [6]:
vbox_text = widgets.VBox([text_0, text_1, my_button, text_2, my_button2])

page = widgets.HBox([vbox_text, vbox_result])
display(page)